In [1]:
import napari_toska as nts
from skimage.morphology import skeletonize
from skimage.data import binary_blobs
from skimage.measure import label
import numpy as np
import napari

In [2]:
viewer = napari.Viewer()

In [3]:
labels = label(binary_blobs(rng=0))
skeleton = skeletonize(labels)

In [4]:
Skeleton = nts.ToskaSkeleton(skeleton, neighborhood='n8')

In [5]:
viewer.add_layer(Skeleton)
viewer.layers[0].features

,labels,object_type
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
143,144,3
144,145,3
145,146,3
146,147,3


In [15]:
from skimage import measure

labelled_skeletons = nts.generate_labeled_skeletonization(skeleton)
parsed_skeletons = nts.parse_all_skeletons(labelled_skeletons, neighborhood='n4')
end_points = measure.label(parsed_skeletons == 1)
branches = measure.label(parsed_skeletons == 2)
branch_points = measure.label(parsed_skeletons == 3)

branches[branches > 0] += end_points.max()
branch_points[branch_points > 0] += end_points.max() + branches.max()

unique_labels = end_points + branches + branch_points

# add object types to features (branch/end/chain)
object_types = np.ones((unique_labels.max()), dtype=int)
object_types[:end_points.max()] = 1
object_types[end_points.max() + 1:branches.max() + end_points.max() + 1] = 2
object_types[-branches.max():] = 3

In [16]:
len(object_types)

148

In [17]:
unique_labels.max()

148